In [ ]:
import random
import math
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from sage.all import Graphics, sphere, line3d, text3d, show

# Define the elements of the cyclic group C3 manually
elements = ['e', 'g', 'g2']
print("Elements of C_3:", elements)

# Function to calculate the entropy of the system as a whole (binary variable)
def calculate_total_system_entropy(probabilities):
    system_prob_true = probabilities['e'] * probabilities['g'] * probabilities['g2']
    system_prob_false = 1 - system_prob_true

    if system_prob_true == 0:
        system_prob_true = 1e-10
    if system_prob_false == 0:
        system_prob_false = 1e-10

    entropy = -system_prob_true * math.log2(system_prob_true) - system_prob_false * math.log2(system_prob_false)
    return entropy

# Function to update probabilities based on the truth states
def update_probabilities(probabilities, influences, truth_states):
    for elem in elements:
        for influenced_elem, influence_value in influences[elem].items():
            if truth_states[elem]:
                probabilities[influenced_elem] = min(1.0, probabilities[influenced_elem] + influence_value)
            else:
                probabilities[influenced_elem] = max(0.0, probabilities[influenced_elem] - influence_value)

# Function to determine the truth states probabilistically based on updated probabilities
def update_truth_states(probabilities, truth_states):
    for elem in elements:
        if random.random() < probabilities[elem]:
            truth_states[elem] = True
        else:
            truth_states[elem] = False

# Function to generate a Cayley diagram where all elements influence each other
def cayley_graph_all_influence(timestep, probabilities, truth_states):
    G = Graphics()

    # Define positions of elements in 3D space (C3 in a triangle configuration)
    positions = {'e': (0, 0, 0), 'g': (1, 0, 0), 'g2': (0.5, 0.866, 0)}
    
    # Add vertices (elements) colored based on their truth state
    for elem, pos in positions.items():
        color = 'white' if truth_states[elem] else 'black'
        G += sphere(pos, size=0.1, color=color)
        G += text3d(elem, pos, fontsize=12)
    
    # Add edges (full cyclic connections)
    for elem1, pos1 in positions.items():
        for elem2, pos2 in positions.items():
            if elem1 != elem2:
                G += line3d([pos1, pos2], color='green', thickness=2)
    
    # Display the current timestep and probabilities in the plot
    G += text3d(f'Timestep {timestep + 1}', (0.2, 0.5, 0.2), fontsize=12, color='blue')
    G += text3d(f'p(e)={probabilities["e"]:.2f}, p(g)={probabilities["g"]:.2f}, p(g2)={probabilities["g2"]:.2f}', 
                (0.2, 0.4, 0.2), fontsize=10, color='red')
    
    show(G)

# Function to calculate the average entropy reduction per timestep
def calculate_avg_entropy_reduction(entropy_values):
    entropy_changes = [entropy_values[i] - entropy_values[i - 1] for i in range(1, len(entropy_values))]
    avg_entropy_reduction = -np.mean(entropy_changes)  # Use the negative mean since entropy is decreasing
    return avg_entropy_reduction

# Function to predict how many timesteps it will take for entropy to reach zero
def predict_time_to_zero_based_on_avg_reduction(initial_entropy, avg_entropy_reduction):
    if avg_entropy_reduction <= 0:
        return float('inf')  # If entropy is not decreasing, we can't predict a time to zero
    return initial_entropy / avg_entropy_reduction

# Widgets for user input
initial_prob_e = widgets.FloatSlider(value=1.0, min=0.0, max=1.0, step=0.01, description='Initial p(e):')
initial_prob_g = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.01, description='Initial p(g):')
initial_prob_g2 = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.01, description='Initial p(g2):')

influence_e_g = widgets.FloatSlider(value=0.1, min=0.0, max=0.2, step=0.01, description='Influence e->g:')
influence_g_g2 = widgets.FloatSlider(value=0.1, min=0.0, max=0.2, step=0.01, description='Influence g->g2:')
influence_g2_e = widgets.FloatSlider(value=0.1, min=0.0, max=0.2, step=0.01, description='Influence g2->e:')

timesteps = widgets.IntSlider(value=10, min=1, max=100, step=1, description='Timesteps:')

# Function to run the simulation and display the results
def run_simulation(steps):
    print("Simulation started!")  # Debugging print statement
    
    probabilities = {'e': initial_prob_e.value, 'g': initial_prob_g.value, 'g2': initial_prob_g2.value}
    influences = {
        'e': {'g': influence_e_g.value},
        'g': {'g2': influence_g_g2.value},
        'g2': {'e': influence_g2_e.value}
    }
    
    truth_states = {'e': True, 'g': False, 'g2': False}
    
    entropy_values = []
    
    for t in range(steps):
        update_probabilities(probabilities, influences, truth_states)
        update_truth_states(probabilities, truth_states)
        
        # Generate and show the Cayley diagram for the current time step
        cayley_graph_all_influence(t, probabilities, truth_states)
        
        # Calculate entropy
        H = calculate_total_system_entropy(probabilities)
        entropy_values.append(H)
    
    avg_entropy_reduction = calculate_avg_entropy_reduction(entropy_values)
    initial_entropy = entropy_values[0]
    time_to_zero_entropy = predict_time_to_zero_based_on_avg_reduction(initial_entropy, avg_entropy_reduction)
    
    # Display results
    print(f"Initial entropy: {initial_entropy:.4f}")
    print(f"Average entropy reduction per timestep: {avg_entropy_reduction:.4f}")
    print(f"Predicted time to zero entropy: {time_to_zero_entropy:.2f} timesteps")

# Function to handle button click event
def on_button_click(b):
    run_simulation(timesteps.value)

# Import the necessary Output widget
from IPython.display import display
import ipywidgets as widgets

# Create an output widget
output = widgets.Output()

# Function to handle button click event and capture output
def on_button_click(b):
    with output:
        output.clear_output()  # Clear previous output before running simulation
        run_simulation(timesteps.value)

# Button to run the simulation
run_button = widgets.Button(description="Run Simulation")
run_button.on_click(on_button_click)




# Display the widgets and output area
display(initial_prob_e, initial_prob_g, initial_prob_g2, influence_e_g, influence_g_g2, influence_g2_e, timesteps, run_button, output)

